In [1]:
import numpy as np
import pandas as pd
import torch
import json
import os
import pickle
from collections import OrderedDict
import psutil
import gc

In [2]:
def print_txt(msg: str):
    print(msg, end = '\r')
    text_file = open("Outputs/input_cnn_preprocessing.txt", "w")
    text_file.write(msg)
    text_file.close()

In [3]:
##directorys
input_path = 'data/scraped_matches/inputs/'
gaia_path = 'data/scraped_matches/gaia_data/'

#players
player_ids = ['p1','p2']
player_winner_dict = {0: 'p1',1: 'p2'}

In [4]:
#load data
with open('data/scraped_matches/parsed_matches/arabia_matches.pkl', 'rb') as f:
    org_matches = pickle.load(f)
matches = org_matches

#load masterdata
with open('masterdata/masterdata_dict.pkl', 'rb') as f:
        masterdata_dict = pickle.load(f)

with open('masterdata/unit_type_dict.pkl', 'rb') as f:
        unit_type_dict = pickle.load(f)        

with open('masterdata/unit_masterdata_dict.pkl', 'rb') as f:
        unit_masterdata_dict = pickle.load(f)      
        
        


In [5]:
match = matches['50453403']
match_id = '50453403'
match

{'match_id': '50453403',
 'fn': '?gameId=50453403&profileId=594394',
 'map': 'Arabia',
 'map_size': 'Tiny',
 'duration': 3445,
 'dataset': 'Definitive Edition',
 'difficulty': 'Hardest',
 'input_fn': 'data/scraped_matches/inputs/50453403.pkl',
 'gaia_fn': 'data/scraped_matches/gaia_data/50453403.pkl',
 'p1_name': 'Zirafo',
 'p2_name': 'Esteban',
 'p1_elo': 975,
 'p2_elo': 1233,
 'p1_civ': 'Vikings',
 'p2_civ': 'Mayans',
 'p1_xpos': 92.0,
 'p2_xpos': 24.0,
 'p1_ypos': 37.0,
 'p2_ypos': 78.0,
 'winner': 0}

In [6]:
with open(matches[match_id]['input_fn'], 'rb') as f:
            inputs = pickle.load(f)

In [7]:
type_dict = {}
for id in inputs:
    input = inputs[id]
    if input['type'] not in type_dict:
        type_dict[input['type']] = input


In [8]:
type_dict.keys()
#start with move_target_build
#inputs

dict_keys(['Queue', 'Research', 'Build', 'Move', 'Gather Point', 'Gather', 'De Autoscout', 'Order', 'Garrison', 'Ungarrison', 'Wall', 'Target', 'Delete', 'Spawn', 'Back To Work', 'Unqueue', 'Repair', 'Reseed', 'Chat', 'Farm Autoqueue', 'Town Bell', 'Stance', 'Pack Trebuchet', 'Formation', 'Resign'])

In [11]:
rel_input_types = ['Build','Move','Target']
timelimit = 1800

input_matchids = []
input_labels = {}

input_level_types = {}

for index,match_id in enumerate(matches):
    #determine winner of each match
    winner = player_ids[matches[match_id]['winner']]
    
    #open inputs
    with open(matches[match_id]['input_fn'], 'rb') as f:
                    inputs = pickle.load(f)
    
    """
    for p_id in player_ids:
        create 2 views per match
    view_id = str(match_id) + '_' + p_id

    if player_winner_dict[match['winner']] == p_id:
        win_label = 1
    else:
        win_label = 0
    
    
    """
    array_list = []
    #init empty arrays of shape map size
    for p_id in player_ids:  
        arrays = {}
        for type in rel_input_types:
            arrays[type] = np.zeros(shape=(120,120))

        for input_id in inputs:
            input = inputs[input_id]


            if input['type'] in rel_input_types and input['ts_seconds']<=timelimit and p_id == input['player']:
                try:
                    #fill values into arrays
                    arrays[input['type']][int(input['y_pos'])-1][int(input['x_pos'])-1] = 1
                except:
                    print(input)


        
        for i in arrays:
            array_list.append(arrays[i])

    md_array =  np.stack(array_list, axis=0)
    tensor = torch.tensor(md_array)

    #persistate tensor
    output = open(f'data/feature_maps/inputs/{match_id}.pkl' , 'wb')
    pickle.dump(tensor, output)
    output.close()

    #safe label and match_id
    input_labels[match_id] = match['winner']#win_label
    input_matchids.append(match_id)

    #input_labels[view_id] = win_label
    #input_matchids.append(view_id)

    if index % 1000 == 0:
        print_txt(f'{index}/{len(matches)} - {tensor.shape} ')


{'ts_seconds': 98, 'timestamp': '0:01:38.912000', 'type': 'Move', 'param': 'None', 'payload': "{'object_ids': [2221]}", 'player': 'p1', 'position': 'Position(x=2.5, y=121.5)', 'x_pos': 2.5, 'y_pos': 121.5}
{'ts_seconds': 165, 'timestamp': '0:02:45.638000', 'type': 'Move', 'param': 'None', 'payload': "{'object_ids': [2214]}", 'player': 'p2', 'position': 'Position(x=2.5, y=121.5)', 'x_pos': 2.5, 'y_pos': 121.5}
{'ts_seconds': 381, 'timestamp': '0:06:21.275000', 'type': 'Move', 'param': 'None', 'payload': "{'object_ids': [1896]}", 'player': 'p2', 'position': 'Position(x=2.5, y=121.5)', 'x_pos': 2.5, 'y_pos': 121.5}
{'ts_seconds': 588, 'timestamp': '0:09:48.688000', 'type': 'Move', 'param': 'None', 'payload': "{'object_ids': [1601]}", 'player': 'p2', 'position': 'Position(x=2.5, y=121.5)', 'x_pos': 2.5, 'y_pos': 121.5}


In [10]:

output = open('data/feature_maps/input_matchids.pkl' , 'wb')
pickle.dump(input_matchids, output)
output.close()
output = open('data/feature_maps/input_labels.pkl' , 'wb')
pickle.dump(input_labels, output)
output.close()


In [11]:
print_txt("check")